I have two codes for motion detection. Test both

In [1]:
from imutils.video import VideoStream
import argparse
import datetime
import imutils
import time
import cv2
import json
import warnings
import numpy as np

In [13]:
video_location = 'Earth_ball.avi'
vs = cv2.VideoCapture(video_location)

In [3]:
# initialize the first frame in the video stream
avg = None

In [14]:

cv2.imshow("uh", accumulated_thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
# resize the frame, convert it to grayscale, and blur it
frame = vs.read()
frame = frame[1]
frame = imutils.resize(frame, width=500)
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)
cv2.imshow("Gray", gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
#main body
accumulated_thresh = np.zeros((281,500))
while True:
    frame = vs.read()
    frame = frame[1]
    text = "unoccupied"
    
    # if the frame could not be grabbed, then we have reached the end
    # of the video
    if frame is None:
        break
    
    # resize the frame, convert it to grayscale, and blur it
    frame = imutils.resize(frame, width=500)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)

    # if the first frame is None, initialize it
    if avg is None:
        avg = gray.copy().astype("float")
        continue

    # compute the absolute difference between the current frame and
    # first frame
    cv2.accumulateWeighted(gray, avg, 0.5)
    frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg))
    thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1]

    # dilate the thresholded image to fill in holes, then find contours
    # on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_TREE,
    	cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    # save the contours into an accumulated thresh image
    accumulated_thresh = cv2.accumulate(thresh, accumulated_thresh)

	# loop over the contours
    for c in cnts:
		# if the contour is too big, ignore it
        if cv2.contourArea(c) < 500 and cv2.contourArea(c) > 200:
		# compute the bounding box for the contour, draw it on the frame,
		# and update the text
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            text = "Occupied"

    # draw the text and timestamp on the frame
    cv2.putText(frame, "Room Status: {}".format(text), (10, 20),
    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    cv2.putText(frame, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"),
    (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)

	# show the frame and record if the user presses a key
    cv2.imshow("Satellite Feed", frame)
    cv2.imshow("accumulated", accumulated_thresh)
    cv2.imshow("Thresh", thresh)
    cv2.imshow("Frame Delta", frameDelta)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()

KeyboardInterrupt: 

#Method 2

In [9]:
# KNN
backSub = cv2.createBackgroundSubtractorKNN()

In [14]:
# MOG2
backSub = cv2.createBackgroundSubtractorMOG2()

In [15]:
while True:
    ret, frame = vs.read()
    if frame is None:
        break

    fgMask = backSub.apply(frame)

    thresh = cv2.dilate(fgMask, None, iterations=2)  # makes bright regions grow
    cnts = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # loop over the contours
    for c in cnts:
        if cv2.contourArea(c) > 200:
            # compute the bounding box for the contour, draw it on the frame
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            text = "Occupied"

    cv2.rectangle(frame, (10, 2), (100, 20), (255, 255, 255), 1)
   
    cv2.imshow('Frame', frame)
    cv2.imshow('Thresh', thresh)
    cv2.imshow('FG Mask', fgMask)

    cv2.waitKey(0)
    
cv2.destroyAllWindows()

KeyboardInterrupt: 